In [1]:
import os
import xarray as xr
import numpy as np
from scipy.ndimage import gaussian_filter
from glob import glob
from tqdm import tqdm
import os

In [2]:
experiment_name = "INALT60.L120-KRS0020"  
data_resolution = "1d" 
datapath = '/gxfs_work/geomar/smomw523/inalt60/data/INALT60.L120-KRS0020/OW-prep/'
meshpath = [datapath + '2_INALT60.L120-KRS0020_mesh_mask.nc']
output_path = "/gxfs_work/geomar/smomw523/smoothed_data/INALT60.L120-KRS0020/"

os.makedirs(output_path, exist_ok=True)

In [3]:
sigma = 3 #9 INALT 60, 3 for INALT20, corresponds to 66.6km  

In [4]:
periods = [
    #(20120101, 20120125), (20120126, 20120219), (20120220, 20120315),
    #(20120316, 20120409),
    (20120410, 20120504),
    #(20120505, 20120529),
    #(20120530, 20120628), (20121027, 20121120), (20121121, 20121215),
    #(20120629, 20120728),
    #(20120729, 20120827), (20120828, 20120926), (20120927, 20121026),
]

In [5]:
grid_vars = {
        "T": "sossheig",
        "U": "vozocrtx",
        "V": "vomecrty"
    }

for (datestart, dateend) in periods:
    print(f"Starting: {datestart}-{dateend} ...")
    
    filename_pattern = f'2_{experiment_name}_{data_resolution}_{datestart}_{dateend}_grid_[TUV].nc' #TUV
    data_in = sorted(glob(datapath + filename_pattern))
    
    for file in tqdm(data_in):
        grid_key = file.split("_grid_")[-1][0]
        var_name = grid_vars.get(grid_key)

        ds = xr.open_dataset(file)

        if var_name in ds:
            dims = ds[var_name].dims
            sigma_tuple = (0, 0, sigma, sigma) if any(d in dims for d in ["depthu", "depthv"]) else (0, sigma, sigma)
            
            smoothed_var = gaussian_filter(ds[var_name], sigma=sigma_tuple)
            ds[var_name] = (dims, smoothed_var)  

            new_filename = os.path.basename(file).replace("_grid_", f"_smoothed_{sigma}_grid_")
            ds.to_netcdf(os.path.join(output_path, new_filename))

        ds.close()

Starting: 20120410-20120504 ...


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [10:06<00:00, 202.30s/it]
